# Nested Chats for Blog Post Creation and Reflection

This notebook demonstrates how to orchestrate nested multi-agent conversations using AutoGen to accomplish a creative writing task. 

#### Workflow: 
1. A `Writer` agent is assigned the task of drafting a concise blogpost on Responsible AI. 
2. The draft is then critiqued by a `Critic` agent, which launches a nested conversation involving multiple specialized reviewer agents (SEO, Legal, Ethics, Meta).
3. The nested evaluation process simulates real-world collaborative content development workflows with minimal human input.

## Setup

In [7]:
import os
import autogen

llm_config = {
    "config_list": [{
        "model": "gpt-4o",
        "api_key": os.environ["OPENAI_API_KEY"], 
    }]
}

## Define a Task

In [12]:
task = '''
        Write a concise but engaging blogpost about
       Responsible AI. Make sure the blogpost is
       within 100 words.
       '''

## Create a Writer Agent

In [13]:
from autogen import AssistantAgent

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [14]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [15]:
print(reply)

**Embracing Responsible AI: A Necessity for the Future**

As Artificial Intelligence permeates various aspects of our daily lives, adopting responsible AI practices is paramount. Responsible AI ensures fairness, transparency, and accountability, crucial for building trust and safeguarding our rights. It involves creating and deploying AI systems that prioritize ethical standards, mitigate bias, and foster inclusivity. Businesses and policymakers must commit to rigorous oversight and collaboration with diverse communities to address potential risks. By harnessing technology ethically and responsibly, we can unlock AI's transformative potential while aligning with societal values, shaping a future where technology enhances rather than infringes upon human dignity.


## Adding Reflection 

Introduce a `Critic` agent whose purpose is to review the output from the `Writer`. It reads the content and gives specific feedback for improvement. This starts a short interaction loop between the Critic and Writer.

In [16]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [17]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       Responsible AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Empowering the Future: The Importance of Responsible AI**

In an era where artificial intelligence is interwoven into the fabric of daily life, responsible AI emerges as a beacon of ethical innovation. It calls for transparency, fairness, and accountability in AI systems to prevent bias and ensure safety. By prioritizing human-centered design and robust regulation, responsible AI fosters trust and societal benefit. It mandates collaboration among governments, businesses, and researchers to create technologies that enhance life without infringing on human rights. As we advance, responsible AI stands as the compass guiding us toward an equitable digital future, maximizing progress with mindfulness and integrity.

------------------

## Nested Chat

To enrich the review process, we create additional specialized reviewer agents:

- `SEO_Reviewer:` Optimizes content for search visibility
- `Legal_Reviewer:` Checks for legal compliance
- `Ethics_Reviewer:` Ensures content is ethically responsible
- `Meta_Reviewer:` Aggregates feedback from all reviewers

Each agent gives its own feedback before the `Meta_Reviewer` delivers the final summary.

In [21]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [22]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [23]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [24]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve task

The `Critic` agent uses the `register_nested_chats()` method to coordinate the nested conversation between all reviewers. A helper function is used to package the original writer’s blogpost for review.

In [25]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [26]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [27]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       Responsible AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Understanding Responsible AI: Ethics in the Digital Age**

As AI technology advances, embracing Responsible AI becomes imperative. It's about creating systems that are ethical, transparent, and accountable. These programs should minimize bias, ensure privacy, and emphasize human welfare. Developers and stakeholders must engage in continuous dialogue to establish fair guidelines and maintain a culture of trust. By prioritizing responsible use, we can harness AI’s potential for societal benefits without compromising our ethical standards. Balancing innovation with responsibility will ensure AI remains a force for good, aligning with humanity's best interests in the digital era.

-----------------------------------------------------

## Get the summary
Retrieve the summary of the revised blog post after the nested review concludes.

In [28]:
print(res.summary)

**Navigating the Landscape of Responsible AI**

Responsible AI is integral to the future, melding innovation with ethics. It's about crafting ethical AI systems that prioritize transparency and accountability, minimizing bias, and promoting human welfare. Emphasizing AI transparency, stakeholders must engage diverse communities to develop inclusive guidelines, ensuring legal compliance with standards like GDPR and CCPA. Instead of guarantees, we aim to minimize bias and reinforce privacy, fostering trust and safeguarding digital integrity. By balancing technological advancement with ethical considerations, Responsible AI steers the future towards equitable and beneficial outcomes for all.
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
